<a href="https://colab.research.google.com/github/arekciminski/model_SW_SNN/blob/Prediction_model/SW_KNN_LSVR_MLP_ANN_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ładowanie bibliotek 

In [2]:
import os.path
import pandas as pd
import seaborn as sns
from joblib import dump, load
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import cross_val_score

#Ładowanie pliku z danymi z Google Drive

In [3]:
file_name='dane_do_uczenia_1_500000.csv'
folder = '/content/drive/MyDrive/Colab Notebooks/final_project/'
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


#Ładowanie pliku lokalnego do pamięci


In [4]:
df= pd.read_csv(folder + file_name,decimal =',',sep=';')

In [5]:
df = df[df['error_output']==0]

In [6]:
df = df.iloc[0:100000,:]

In [7]:

df.shape

(100000, 22)

In [8]:
df.head()

,Unnamed: 0,error_output,time_output,head_output_001,head_output_002,head_output_055,head_output_083,head_output_158,head_output_180,flow_output_F1,...,tank_output_180,energy_output_F1,energy_output_K1,energy_output_P1,tank_input_180,pump_input_F1,pump_input_K1,pump_input_P1,demand_input_par2,demand_input_kar2
3,3,0,0,171.688080,204.964844,204.508545,204.187195,204.079819,169.699997,137.287430,...,169.580811,21.494255,30.574085,7.613600,3.70,0.73,0.82,0.81,1.35,1.38
8,8,0,0,171.587021,181.432541,181.311340,181.065262,180.895660,167.619995,199.358078,...,167.838211,31.701595,6.275290,2.028228,1.62,0.80,0.51,0.58,1.03,1.05
12,12,0,0,170.038803,203.679108,203.759109,203.419128,203.280502,168.229996,130.457672,...,168.356750,18.574577,11.199346,14.652009,2.23,0.71,0.76,0.96,1.26,1.06
13,13,0,0,177.310242,196.880997,196.835800,196.622559,196.429016,169.710007,283.203064,...,170.197037,52.643723,9.536500,7.073823,3.71,1.00,0.67,0.77,0.77,0.91
14,14,0,0,170.258987,202.284668,202.366318,202.218246,201.977463,169.259995,94.678185,...,169.416534,12.031251,7.318162,8.812004,3.26,0.70,0.73,0.83,0.67,0.55


#Losowa zmiana pozycji wierszy w df


In [9]:
df = df.sample(frac = 1)

#Podział danych na X i y

In [10]:
df.columns

Index(['Unnamed: 0', 'error_output', 'time_output', 'head_output_001',
       'head_output_002', 'head_output_055', 'head_output_083',
       'head_output_158', 'head_output_180', 'flow_output_F1',
       'flow_output_K1', 'flow_output_P1', 'tank_output_180',
       'energy_output_F1', 'energy_output_K1', 'energy_output_P1',
       'tank_input_180', 'pump_input_F1', 'pump_input_K1', 'pump_input_P1',
       'demand_input_par2', 'demand_input_kar2'],
      dtype='object')

In [11]:
'''x_columns =['tank_input_180',  'head_output_001',
            'head_output_002', 'head_output_055', 'head_output_083',
            'head_output_158', 'head_output_180', 'pump_input_P1',
            'demand_input_par2', 'demand_input_kar2','flow_output_F1',
            'flow_output_K1', 'flow_output_P1']
y_column = ['pump_input_F1', 'pump_input_K1', 'pump_input_P1']'''

"x_columns =['tank_input_180',  'head_output_001',\n            'head_output_002', 'head_output_055', 'head_output_083',\n            'head_output_158', 'head_output_180', 'pump_input_P1',\n            'demand_input_par2', 'demand_input_kar2','flow_output_F1',\n            'flow_output_K1', 'flow_output_P1']\ny_column = ['pump_input_F1', 'pump_input_K1', 'pump_input_P1']"

In [12]:
x_columns =['tank_input_180', 'head_output_180','head_output_083',
            'head_output_158', 'demand_input_par2', 'demand_input_kar2']
y_column = ['head_output_001', 'head_output_002','head_output_055']

In [13]:
X = df[x_columns]
y = df[y_column]
y

,head_output_001,head_output_002,head_output_055
182417,173.751434,189.113159,189.189682
224061,169.317795,188.320587,188.406219
9662,176.448822,216.041824,215.658905
170552,172.845459,200.446014,200.401886
254412,174.287308,200.497070,200.695190
...,...,...,...
88166,175.102997,203.723709,203.695602
51580,174.815842,204.182327,204.137115
170793,175.586731,203.440628,203.365692
287759,174.050125,182.681580,182.358139


#Podział danych na testowe i treningowe

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [15]:
# example of making a prediction with the direct multioutput regression model
from sklearn.datasets import make_regression
from sklearn.multioutput import MultiOutputRegressor, RegressorChain
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.svm import LinearSVR
from sklearn.neighbors import KNeighborsRegressor


#Linear SVR Regressor

In [16]:
# define dataset
# define base model
model = LinearSVR()
# define the direct multioutput wrapper model
MOR = MultiOutputRegressor(model)


In [17]:
# fit the model on the whole dataset
MOR.fit(X_train, y_train)


/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


MultiOutputRegressor(estimator=LinearSVR())

In [18]:
# make a single prediction
y_pred_MOR = MOR.predict(X_test)

#for i in range(len(y_pred_MOR)):
#  print(y_pred_MOR[i][6],y_test['error_output'][i])
y_pred_MOR[0]

array([173.53599111, 214.60532288, 214.38416083])

In [19]:
y_test

,head_output_001,head_output_002,head_output_055
235199,172.029373,214.848816,214.400192
189106,175.432846,202.079132,201.938171
253799,173.924072,193.520706,193.684647
77974,173.986023,210.555450,210.379745
149054,174.903168,188.570953,188.042130
...,...,...,...
169079,174.143173,200.219421,200.292648
20649,175.963654,193.063980,192.989151
133251,172.855133,202.082504,201.564087
274929,172.716583,194.928345,194.705872


In [20]:
MSE_test_LR = mean_squared_error(y_test, y_pred_MOR)
r2_test_LR = r2_score(y_test, y_pred_MOR)
[MSE_test_LR, r2_test_LR]

[1.7254690606764278, 0.6903223070624952]

In [21]:
dump(MOR, folder + 'MOR_best.joblib')

['/content/drive/MyDrive/Colab Notebooks/final_project/MOR_best.joblib']

#KNN Regresor

In [22]:
param_grid_knr ={
    'n_neighbors' : list(range(2,10,2)),
    'algorithm' : ['ball_tree' , 'kd_tree'],
}

In [23]:
KNR = KNeighborsRegressor()

In [24]:

#KNR_reg_GS = GridSearchCV(KNR, param_grid_knr,verbose=2,n_jobs=1)

#KNR_reg_GS.fit(X_train, y_train)

In [25]:
best_params_ = {'algorithm': 'ball_tree', 'n_neighbors': 8}


In [26]:
#KNR_best = KNeighborsRegressor(n_neighbors = KNR_reg_GS.best_params_['n_neighbors'],algorithm = KNR_reg_GS.best_params_['algorithm'])

In [27]:
KNR_best = KNeighborsRegressor(n_neighbors = best_params_['n_neighbors'],algorithm = best_params_['algorithm'])

In [28]:
KNR_best.fit(X_train,y_train)

KNeighborsRegressor(algorithm='ball_tree', n_neighbors=8)

In [29]:
y_pred_KNR = KNR_best.predict(X_test)

In [30]:
MSE_test_KNR = mean_squared_error(y_test, y_pred_KNR)
r2_test_KNR = r2_score(y_test, y_pred_KNR)
[MSE_test_KNR, r2_test_KNR]

[1.6179588790861201, 0.7084421244280198]

In [31]:

dump(KNR_best, folder + 'KNR_best.joblib')

['/content/drive/MyDrive/Colab Notebooks/final_project/KNR_best.joblib']

#MLP Regressor


In [32]:
from sklearn.neural_network import MLPRegressor
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler 

In [33]:
scaler = StandardScaler()  
scaler.fit(X_train)  
X_train_scale = scaler.transform(X_train)  
X_test_scale = scaler.transform(X_test)

In [34]:
param_grid_mlp ={
        'hidden_layer_sizes' : list(range(10,100,20)),
        'activation' : ['identity', 'relu'],
}


In [35]:
MLP_reg = MLPRegressor(max_iter=5000)

MLP_reg_GS = GridSearchCV(MLP_reg, param_grid_mlp,verbose=2)

MLP_reg_GS.fit(X_train_scale, y_train)


Fitting 5 folds for each of 10 candidates, totalling 50 fits
[CV] END .........activation=identity, hidden_layer_sizes=10; total time=   5.0s
[CV] END .........activation=identity, hidden_layer_sizes=10; total time=   3.3s
[CV] END .........activation=identity, hidden_layer_sizes=10; total time=   3.7s
[CV] END .........activation=identity, hidden_layer_sizes=10; total time=   3.5s
[CV] END .........activation=identity, hidden_layer_sizes=10; total time=   5.4s
[CV] END .........activation=identity, hidden_layer_sizes=30; total time=   3.6s
[CV] END .........activation=identity, hidden_layer_sizes=30; total time=   4.0s
[CV] END .........activation=identity, hidden_layer_sizes=30; total time=   5.6s
[CV] END .........activation=identity, hidden_layer_sizes=30; total time=   2.9s
[CV] END .........activation=identity, hidden_layer_sizes=30; total time=   4.1s
[CV] END .........activation=identity, hidden_layer_sizes=50; total time=   3.7s
[CV] END .........activation=identity, hidden_la

GridSearchCV(estimator=MLPRegressor(max_iter=5000),
             param_grid={'activation': ['identity', 'relu'],
                         'hidden_layer_sizes': [10, 30, 50, 70, 90]},
             verbose=2)

In [60]:
best_params_ = {'activation': 'identity','hidden_layer_sizes': 100}
best_params_ = MLP_reg_GS.best_params_
best_params_

{'activation': 'relu', 'hidden_layer_sizes': 30}

In [37]:
MLP_reg_GS_best = MLPRegressor(max_iter=500,verbose=1,hidden_layer_sizes = best_params_['hidden_layer_sizes'],activation = best_params_['activation'])

In [38]:
dump(MLP_reg_GS_best, folder + 'MLP_reg_GS_best.joblib')

['/content/drive/MyDrive/Colab Notebooks/final_project/MLP_reg_GS_best.joblib']

In [39]:
MLP_reg_GS_best.fit(X_train_scale,y_train)

Iteration 1, loss = 16981.43525602
Iteration 2, loss = 12386.82644608
Iteration 3, loss = 6137.59978844
Iteration 4, loss = 2400.81755341
Iteration 5, loss = 1108.37222863
Iteration 6, loss = 726.30050592
Iteration 7, loss = 552.83347575
Iteration 8, loss = 441.00273288
Iteration 9, loss = 362.33953825
Iteration 10, loss = 301.16747112
Iteration 11, loss = 248.22724918
Iteration 12, loss = 199.61498319
Iteration 13, loss = 154.83898849
Iteration 14, loss = 115.17454931
Iteration 15, loss = 82.23605036
Iteration 16, loss = 56.98362437
Iteration 17, loss = 39.18044625
Iteration 18, loss = 27.30828929
Iteration 19, loss = 19.52520660
Iteration 20, loss = 14.34866911
Iteration 21, loss = 10.78899949
Iteration 22, loss = 8.26861635
Iteration 23, loss = 6.45284633
Iteration 24, loss = 5.14567591
Iteration 25, loss = 4.19456214
Iteration 26, loss = 3.48635558
Iteration 27, loss = 2.93372332
Iteration 28, loss = 2.48534124
Iteration 29, loss = 2.10201070
Iteration 30, loss = 1.76465526
Iterati

MLPRegressor(hidden_layer_sizes=30, max_iter=500, verbose=1)

In [40]:
y_pred_MLP = MLP_reg_GS_best.predict(X_test_scale)

In [41]:
MSE_test_MLP = mean_squared_error(y_test, y_pred_MLP)
r2_test_MLP = r2_score(y_test, y_pred_MLP)
[MSE_test_KNR,r2_test_MLP]

[1.6179588790861201, 0.7404749870246464]

#Decission Tree Regresor

In [42]:
dt_reg = DecisionTreeRegressor(random_state=0)

In [43]:
param_grid_dt ={
    'max_depth' : list(range(2,21,2)),
    'min_samples_split' : list(range(2,21,2)),
}

In [44]:
dt_reg_GS = GridSearchCV(dt_reg, param_grid_dt,verbose=2,n_jobs=1)

dt_reg_GS.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
[CV] END ...................max_depth=2, min_samples_split=2; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=2; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=2; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=2; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=2; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=4; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=4; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=4; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=4; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=4; total time=   0.1s
[CV] END ...................max_depth=2, min_samples_split=6; total time=   0.1s
[CV] END ...................max_depth=2, min_s

GridSearchCV(estimator=DecisionTreeRegressor(random_state=0), n_jobs=1,
             param_grid={'max_depth': [2, 4, 6, 8, 10, 12, 14, 16, 18, 20],
                         'min_samples_split': [2, 4, 6, 8, 10, 12, 14, 16, 18,
                                               20]},
             verbose=2)

In [61]:
best_params_ = {'max_depth': dt_reg_GS.best_params_['max_depth'], 'min_samples_split' : dt_reg_GS.best_params_['min_samples_split']}
print(best_params_)
best_params_= dt_reg_GS.best_params_



{'max_depth': 8, 'min_samples_split': 14}


In [46]:

dt_reg = DecisionTreeRegressor(random_state=0,max_depth = best_params_['max_depth'],min_samples_split = best_params_['min_samples_split'])

In [47]:
y_pred_DT = dt_reg_GS.predict(X_test)

In [48]:
MSE_test_DT = mean_squared_error(y_test, y_pred_DT)
r2_test_DT = r2_score(y_test, y_pred_DT)
[MSE_test_DT, r2_test_DT]

[1.4865855361855658, 0.7412173466160162]

In [49]:
dump(dt_reg_GS, folder + 'DT_best_pred_model.joblib')

['/content/drive/MyDrive/Colab Notebooks/final_project/DT_best_pred_model.joblib']

In [50]:
y_pred_DT[0]

array([174.64733226, 214.35595809, 214.12361537])

In [51]:
y_test.iloc[0,:]

head_output_001    172.029373
head_output_002    214.848816
head_output_055    214.400192
Name: 235199, dtype: float64

#ANN

In [52]:
from keras import models
from keras import layers
from keras.layers import Dropout

In [53]:
len(y_train.columns)

3

In [54]:
ANN = models.Sequential()
ANN.add(layers.Dense(512, activation='elu', input_shape=(len(X_train_scale[0]),)))
ANN.add(Dropout(0.2))
ANN.add(layers.Dense(256, activation='elu'))
ANN.add(layers.Dense(len(y_train.columns), activation='relu'))

In [55]:
ANN.compile(optimizer="adam",loss='mean_squared_error')

In [56]:
ANN_reg_best = ANN.fit(X_train_scale, y_train, validation_data=(X_test_scale, y_test), epochs=100, batch_size=64)

Epoch 1/100
1094/1094 [==============================] - 10s 4ms/step - loss: 2785.2351 - val_loss: 2.3301
Epoch 2/100
1094/1094 [==============================] - 4s 4ms/step - loss: 24.2422 - val_loss: 2.3619
Epoch 3/100
1094/1094 [==============================] - 4s 4ms/step - loss: 19.5939 - val_loss: 2.6570
Epoch 4/100
1094/1094 [==============================] - 5s 4ms/step - loss: 17.1182 - val_loss: 2.0183
Epoch 5/100
1094/1094 [==============================] - 4s 4ms/step - loss: 15.1534 - val_loss: 2.2973
Epoch 6/100
1094/1094 [==============================] - 4s 4ms/step - loss: 13.6604 - val_loss: 2.2093
Epoch 7/100
1094/1094 [==============================] - 5s 4ms/step - loss: 11.8938 - val_loss: 1.8085
Epoch 8/100
1094/1094 [==============================] - 4s 4ms/step - loss: 10.5092 - val_loss: 1.6913
Epoch 9/100
1094/1094 [==============================] - 4s 4ms/step - loss: 8.8183 - val_loss: 2.2566
Epoch 10/100
1094/1094 [==============================] - 4s 4

In [57]:
dump(ANN, folder + 'ANN_reg_best.joblib')

['/content/drive/MyDrive/Colab Notebooks/final_project/ANN_reg_best.joblib']

In [58]:
y_pred_ANN = ANN_reg_best.model.predict(X_test_scale)

938/938 [==============================] - 1s 1ms/step


In [59]:
MSE_test_ANN = mean_squared_error(y_test, y_pred_ANN)
r2_test_ANN = r2_score(y_test, y_pred_ANN)
[MSE_test_ANN, r2_test_ANN]

[1.4690872439088147, 0.7389844101693955]